In [3]:
pip install librosa

     |████████████████████████████████| 214 kB 3.6 MB/s 
     |████████████████████████████████| 377 kB 49.4 MB/s 
     |████████████████████████████████| 1.2 MB 41.5 MB/s 
     |████████████████████████████████| 56 kB 3.9 MB/s 
  Created wheel for audioread: filename=audioread-3.0.0-py3-none-any.whl size=23706 sha256=72b26866e041bf1d9a14339f624f0129e3ae2674e7f7e80aca2fdab03ed8830c
  Stored in directory: /Users/yulunwu/Library/Caches/pip/wheels/e4/76/a4/cfb55573167a1f5bde7d7a348e95e509c64b2c3e8f921932c3
Successfully built audioread
Note: you may need to restart the kernel to use updated packages.


In [1]:
'''
Please install this specific version of resampy for librosa to work without errors.
'''

'\nPlease install this specific version of resampy for librosa to work without errors.\n'

In [2]:
pip install resampy==0.3.1

     |████████████████████████████████| 3.1 MB 2.5 MB/s 
Note: you may need to restart the kernel to use updated packages.


In [4]:
import soundfile
import os
import glob
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import librosa
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report
import warnings; warnings.filterwarnings('ignore')

OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [5]:
emotions ={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

### Data for binary classification


In [7]:
def load_extract_features(data_path):

    '''
    load_extract_features() is a function that is used to load all the audio files one at a time, compute their features and return the features as well as the target values.

    There are around 8-10 audio files which are corrupted. We hardcode zero values for such files in order to maintain consistency.

    ['calm', 'happy'] emotion data is categorized into 'positive' and  ['angry', 'fearful'] into 'negative'

    Returns:
    1. Features
    2. Binary Target Values
    '''
    final_features,target_emotions, binary_label = [],[], []
    count = 0
    
    for i in glob.glob(data_path + "/Actor_*/*.wav"): #Loop to read every file.
        
        name = os.path.basename(i)
        #We split the name of the file to understand the emotion associated with the file.
        split = name.split("-")
        #We know that the third identifier is associated with the emotion of the audio file. Hence, we use [2] as it represents the third identifier.
        emotion = emotions[split[2]]

        #Below is the code to categorize the emotions into two classes to make this a binary problem.
        if emotion in ['calm', 'happy']:
            binary_label.append(0)
        elif emotion in ['angry', 'fearful']:
            binary_label.append(1)
        else:
            continue
        
        with soundfile.SoundFile(i) as audio:
            waveform = audio.read(dtype="float32")
            sr = audio.samplerate
            
            #Below is the code to extract the Mel spectrogram features
            #128 is the standard for machine learning applications using Mel spectrograms
            m_feature = librosa.feature.melspectrogram(y=waveform, sr=sr, n_mels=128, fmax=sr / 2.0).T
            melspectrogram = np.mean(m_feature,axis=0)
            if melspectrogram.shape != (128,):
                melspectrogram = np.zeros(128)
            
            #Below is the code to extract the chromagram features
            stft_wave = librosa.stft(waveform)
            stft = np.abs(stft_wave)
            c_feature = librosa.feature.chroma_stft(S=stft, sr=sr).T
            chromagram = np.mean(c_feature,axis=0)
            
            #12 is the number of pitch classes
            if chromagram.shape != (12,):
                chromagram = np.zeros(12)
                
            features=np.array([])
            features=np.hstack((chromagram, melspectrogram))
        
            final_features.append(features)
            target_emotions.append(emotion)
            
            count += 1
            if count % 100 == 0:
                print("Processed Audio File Number: ", count)
    
    #We return the features and the binary target values.
    return np.array(final_features), np.array(binary_label)

In [ ]:
#Please change the path below to the path of the folder saved in your computer.
data_path = './Audio_Speech_Actors_01-24'
X, binary_label = load_extract_features(data_path)

In [9]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import statistics
import scipy.io as sp
import functools as f
from scipy import stats as sps

In [15]:
# Note that I didn't use the raw dataset, I use the csv file posted on Piazza by some kind classmate
df = pd.read_csv('features.csv', sep=',',header=None)

In [16]:
df

0         1         2         3         4         5         6    \
0    1.0  0.552245  0.504338  0.512778  0.550540  0.572176  0.615567   
1    1.0  0.550732  0.508484  0.466810  0.438485  0.428218  0.445352   
2    1.0  0.672768  0.601451  0.548317  0.477806  0.453192  0.516665   
3    1.0  0.602309  0.605448  0.558788  0.529441  0.546190  0.568049   
4    0.0  0.624107  0.559042  0.513058  0.526347  0.590221  0.618584   
..   ...       ...       ...       ...       ...       ...       ...   
763  1.0  0.493344  0.468308  0.472621  0.484694  0.531912  0.626702   
764  0.0  0.664177  0.633587  0.590743  0.601933  0.623831  0.643299   
765  0.0  0.697806  0.658317  0.614802  0.639169  0.717383  0.743908   
766  0.0  0.628023  0.596604  0.643935  0.655228  0.659819  0.725635   
767  0.0  0.630163  0.652205  0.637307  0.684746  0.764457  0.795282   

          7         8         9    ...           131           132  \
0    0.603708  0.573230  0.593454  ...  2.407505e-04  1.812392e-04   
1    0.444528  0.444478  0.500419  ...  4.505552e-05  4.551989e-05   
2    0.570100  0.599750  0.647746  ...  3.031455e-04  2.504395e-04   
3    0.566027  0.523464  0.526789  ...  1.048496e-04  8.544709e-05   
4    0.620928  0.614490  0.596036  ...  2.395205e-04  1.650887e-04   
..        ...       ...       ...  ...           ...           ...   
763  0.712453  0.772570  0.750475  ...  5.407017e-05  4.915729e-05   
764  0.671203  0.668196  0.683204  ...  3.044766e-06  2.261041e-06   
765  0.781170  0.749796  0.722322  ...  1.015495e-06  6.037960e-07   
766  0.808151  0.814369  0.771093  ...  2.175136e-07  1.312530e-07   
767  0.819001  0.802949  0.774911  ...  4.666486e-07  2.649286e-07   

              133           134           135           136           137  \
0    1.270704e-04  8.945510e-05  5.687699e-05  2.774789e-05  1.360746e-05   
1    3.664347e-05  2.707523e-05  1.896371e-05  1.495736e-05  7.513548e-06   
2    2.244678e-04  1.305480e-04  1.123456e-04  8.232176e-05  6.906760e-05   
3    5.829688e-05  5.944907e-05  4.806442e-05  3.390669e-05  2.690935e-05   
4    6.797433e-05  4.822332e-05  3.622557e-05  1.489366e-05  5.819597e-06   
..            ...           ...           ...           ...           ...   
763  3.140714e-05  2.459202e-05  1.671288e-05  1.249650e-05  1.220924e-05   
764  1.881779e-06  2.103687e-06  2.280329e-06  1.940662e-06  1.277455e-06   
765  4.062441e-07  3.169898e-07  2.241397e-07  1.854334e-07  1.122429e-07   
766  7.179825e-08  7.687991e-08  6.666074e-08  4.827929e-08  4.940794e-08   
767  1.989734e-07  1.265435e-07  7.476335e-08  1.109522e-07  7.461457e-08   

              138           139           140  
0    6.994450e-06  9.212427e-07  8.153847e-09  
1    3.295148e-06  3.120650e-07  7.757001e-09  
2    2.829970e-05  2.610200e-06  1.508926e-08  
3    1.398858e-05  1.212124e-06  1.013364e-08  
4    2.780916e-06  4.201133e-07  8.359251e-09  
..            ...           ...           ...  
763  7.274141e-06  6.256171e-07  1.079954e-08  
764  5.335462e-07  4.841995e-08  8.426449e-09  
765  5.348774e-08  1.107998e-08  7.719827e-09  
766  2.523429e-08  8.708965e-09  7.872195e-09  
767  3.264304e-08  9.176199e-09  7.723372e-09  

[768 rows x 141 columns]

The first column of this dataset is the label, and the rests are features.

In [363]:
# Separate dataset into features(X) and labels(Y)
X = df.iloc[:,1:]
Y = df.iloc[:,0].values.reshape(X.shape[0],1)
print(X.shape)
print(Y.shape)

(768, 140)
(768, 1)


In [190]:
# Replace all label 0 with label -1 for convience
Y = np.where((Y == 0),-1,Y)

In [192]:
# Separate data into training and testing set
X_train, X_test,Y_train,Y_test = train_test_split(X,Y, test_size=0.3,
 random_state=10)

In [193]:
print(X_train.shape)
print(X_test.shape)

(537, 140)
(231, 140)


In [340]:
def gradient_descent(x,y,w,b,lambda0,aita):
    """ Return tuned (b,w) using gradient descent
    Args:
    - x (dataframe (Shape: (N, D))): A NxD dataframe contains N input, each input has D features.
    - y (ndarray (Shape: (N, 1))): A Nx1 column vector corresponding to the acutal labels of x.
    - w (ndarray (Shape: (D, 1))): A Dx1 column vector of initial weights.
    - b (float): A float of initial bias.
    - lambda0 (float): The regularization parameter.
    - aita (float): The learning rate.
    
    Output:
    - (b,w) ((float,ndarray) (Shape: (1,(D, 1))).
    """
    n = x.shape[0]
    zeros = np.ones((n, 1))
    ones = np.ones((n, 1))
    # Two stop criteria
    gradientw_diff = 0.01
    w_diff = 0.000001
    # pad x with 1 to left
    #pad_x = np.hstack((ones,x))
    # w = (b,w)^T
    #w_and_b = np.hstack((b,w))
    gw = 1
    w_last = w - 0.2
    b_array = np.empty((n,1))
    b_array.fill(b)
    # counter = 0
    for j in range(500):
    #while(np.linalg.norm(w-w_last,axis=0)>=w_diff and np.linalg.norm(gw)>=gradientw_diff):
        xw_array = (x@w).to_numpy().reshape(x.shape[0],1)
        hinge = np.maximum(zeros,ones-np.multiply(y,xw_array-b_array))
        gw = 0
        gb = 0
        for i in range(n):
            if(hinge[i] == 0):
                gw += lambda0*w
            else:
                gw += lambda0*w - y[i]*x.iloc[i]
                gb -= y[i]
        gw = gw/n
        gb = gb/n
        w_last = w
        b = b - aita*gb
        b_array.fill(b[0])
        w = w - aita*gw
        # counter +=1
        # This is used if choosing to use while loop
        #if (counter%100 == 0):
            # print(counter)
    return (b,w)

In [254]:
def GD_predict(x,w,b):
    """ Return the ndarray of predicted labels.
    Args:
    - x (dataframe (Shape: (N, D))): A NxD dataframe contains N input, each input has D features.
    - w (ndarray (Shape: (D, 1))): A Dx1 column vector of weights.
    - b (float): A float of initial bias.
    
    Output:
    - (ndarray (Shape: (N, 1))).
    """
    b_array = np.empty((x.shape[0],1))
    b_array.fill(b[0])
    xw_array = (x@w).to_numpy().reshape(x.shape[0],1)
    return np.sign(xw_array+b_array)

In [24]:
# This function takes two list, compare them element wise and return the 
# accuracy
def accuracy(l1, l2):
    return sum(list(map(lambda a,b: 1 if a == b else 0, l1, l2)))/len(l1)

## SVM

In [341]:
lambda0 = 0.5 # regularization parameter
#aita = 0.000001
aita = 0.01 # learning rate

In [342]:
# Initial b and w
b0 = 0
w0 = np.zeros(X_train.shape[1])

In [343]:
# call gradient_descent function to compute b and w
(b,w) = gradient_descent(X_train,Y_train,w0,b0,lambda0,aita)

In [344]:
# Predict label of training set
pred_Y_train = GD_predict(X_train,w,b)

In [345]:
# Compute training accuracy
accuracy_train = accuracy(Y_train,pred_Y_train)

In [352]:
print("The training accuracy for prediction using SVM is",accuracy_train)

The training accuracy for prediction using SVM is 0.6219739292364991


In [348]:
# Predict label of testing set
pred_Y_test = GD_predict(X_test,w,b)

In [349]:
# Compute testing accuracy
accuracy_test = accuracy(Y_test,pred_Y_test)

In [351]:
print("The testing accuracy for prediction using SVM is",accuracy_test)

The testing accuracy for prediction using SVM is 0.6406926406926406


## PCA

In [290]:
def PCA(x):
    """ Return (info_capture,mu,eigenVec[:,:k]), where info_capture indicates how much percentage 
    of information is captured by the reduced model, mu is the mean vector of x, eigenVec[:,:k] 
    is the ndarray contains k eigenvectors associated with k highest eigenvalues.
    Args:
    - x (dataframe (Shape: (N, D))): A NxD dataframe contains N input, each input has D features.
    
    Output:
    - ((float,ndarray,ndarray) (Shape: (1,(D,1),(D,K)))).
    """
    n = x.shape[0]
    mu = np.mean(x,axis=0)
    sigma = ((n-1)/n)*np.cov(x,rowvar=False)
    eigenVal,eigenVec = np.linalg.eig(sigma)
    # np.linalg.eig documentation doesn't mention whether 
    sorted_index = np.argsort(-eigenVal)
    eigenVal = eigenVal[sorted_index]
    eigenVec = eigenVec[:,sorted_index]
    sum_eigenVal = sum(eigenVal)
    k = 1
    sum_k_sum_eigenVal = eigenVal[k-1]
    info_capture = sum_k_sum_eigenVal/sum_eigenVal
    while(info_capture<0.95):
        k += 1
        sum_k_sum_eigenVal += eigenVal[k-1]
        info_capture = sum_k_sum_eigenVal/sum_eigenVal
    return (info_capture,mu,eigenVec[:,:k])



In [305]:
def PCA_predict(x,eigenVec,mu):
    return (x-mu)@eigenVec

In [299]:
# Call PCA to compute PCA parameter
(info_capture,mu,eigenVec) =  PCA(X_train)

In [306]:
# Reduce the dimension of training set using PCA parameter
X_train_PCA = PCA_predict(X_train,eigenVec,mu)

In [353]:
lambda0 = 0.3 # regularization parameter
#aita = 0.000001
aita = 0.001 # learning rate

In [354]:
# Initial b and w
b0_PCA = 0
w0_PCA = np.zeros(X_train_PCA.shape[1])

In [355]:
# call gradient_descent function to compute b and w with low dimensional training set
(b_PCA,w_PCA) = gradient_descent(X_train_PCA,Y_train,w0_PCA,b0_PCA,lambda0,aita)

In [356]:
# Predict label of low dimensional training set
pred_Y_train_PCA = GD_predict(X_train_PCA,w_PCA,b_PCA)

In [357]:
# Compute training accuracy
accuracy_train_PCA = accuracy(Y_train,pred_Y_train_PCA)

In [358]:
print("The training accuracy for prediction using SVM after reducing dimension using PCA is",accuracy_train_PCA)

The training accuracy for prediction using SVM after reducing demension using PCA is 0.6778398510242085


In [359]:
# Reduce the dimension of testing set using PCA parameter
X_test_PCA = PCA_predict(X_test,eigenVec,mu)

In [360]:
# Predict label of low dimensional testing set
pred_Y_test_PCA = GD_predict(X_test_PCA,w_PCA,b_PCA)

In [361]:
# Compute testing accuracy
accuracy_test_PCA = accuracy(Y_test,pred_Y_test_PCA)

In [362]:
print("The testing accuracy for prediction using SVM after reducing dimension using PCA is",accuracy_test_PCA)

The testing accuracy for prediction using SVM after reducing demension using PCA is 0.5887445887445888
